In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Make given files into pandas**

In [ ]:
import numpy as py
import pandas as pd

base_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

# **1. Exploring Data**

**Observe data state**

In [ ]:
train.shape, test.shape

above result of the code shows as shape as (row,colum)

**Explore data**

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
test.shape

In [ ]:
test.head()

As observing two datasets, there are different number in feature.
To know common feature, use below code.

In [ ]:
test_head = set(list(test.head(0).columns))
print(test_head)
train_head = set(list(train.head(0).columns))
print(train_head)

In [ ]:
common_feature = list(test_head&train_head) #intersection of the given two data
print(len(common_feature))
print(common_feature)

In [ ]:
submission.shape

In [ ]:
submission.head()

**Explore General structure of data**

In [ ]:
train.info() # through this code, you know amount of missing values 
            # for each columns in DataFrame and which kinds of data types in it

According to above information, it says Non-Null count. 
Non-Null count means there is no missing value. 
Missing value is the value that the value does not exist in the data. (= 누락값) 
- object: str data type

In [ ]:
test.info()

# 2. Feature Engineering

What is Feature engineering

To observe data as visualizing data, we need to convert data into available data for visualization. 

ex. in this case,

The given 'datetime' in train dataset is year-month-day hour:minute:second.
So, we have to convert its data available shape for visiualization.

In [ ]:
train.head()

[code]

dt = 'datetime'

date = 'date'

y = 'year'

mo = 'month'

day = 'day'

h = 'hour'

mi = 'minute'

s = 'second'


train[date] = train[dt].apply(lambda x: x.split()[0])

train[y] = train[dt]

-----------------
train['year'] = train['datetime'].apply(lambda x: x.split()[0].split['-'][0])

--> plz see book as you want to know this syntax meaning

x in this case is string in datetime in train dataset.

In [ ]:
first,second = ['year','month','day','-'],['hour','minute','second',':']

for i in [first,second]:
    standard = i[-1]
    split = 0 if standard == '-' else 1
    for j in range(len(i)-1):
        train[i[j]] = train['datetime'].apply(lambda x: x.split()[split].split(standard)[j])


In [ ]:
train.head()

**Create Day of week as feature**

# 3. Visulzation of data

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib inline # which returns graph from matplot in jupiter notebook but in this kaggle, does not need

## **Distribution plot analysis**

In [ ]:
mpl.rc('font',size =10)
sns.displot(train['count'])

x axis = target value

y axis = amount of target value

In [ ]:
sns.displot(np.log(train['count']))

## **barplot analysis**

step 1: prepare m x n matrix

In [ ]:
mpl.rc('font',size = 14) # font size
mpl.rc('axes', titlesize = 15) # size control for each axises
figure, axes = plt.subplots(nrows = 3, ncols = 2) # assign 3 x 2 matrix
plt.tight_layout() # make white space between graphs
figure.set_size_inches(10,9) # make whole size of figure into 10 x 9 inch

step 2: assign subplots into each axes

In [ ]:
features = first[:-1] + second[:-1]
print(features)

In [ ]:
row_col = [[i,j] for i in range (0,3) for j in range (0,2)]
print(row_col)
for i,x_ in enumerate(features):
    sns.barplot(x = x_, y = 'count', data = train, ax = axes[row_col[i][0],row_col[i][1]])

In [ ]:
figure

Step 3: Detailed setting

Add title and make white spaces between numbers

In [ ]:
for i,j in enumerate(row_col):
    axes[j[0],j[1]].set(title = 'Rental amounts by '+ features[i])

In [ ]:
figure

In [ ]:
axes[1,0].tick_params(axis = 'x', labelrotation = 90)
axes[1,1].tick_params(axis = 'x', labelrotation = 90)

In [ ]:
figure

## **Box plot analysis**

In [ ]:
x_features = ['season','weather','holiday','workingday']
axes_ = [[i,j] for i in range(0,2) for j in range(0,2)]

figure,axes = plt.subplots(nrows=2,ncols=2)
plt.tight_layout()
figure.set_size_inches(10,10)

for i_, x_ in enumerate(x_features):
    axe = axes_[i_]
    sns.boxplot(x=x_,y='count',data=train, ax=axes[axe[0],axe[1]])
    axes[axe[0],axe[1]].set(title='Box Plot On Count Across '+ x_)

# axes[0, 1].tick_params(axis='x', lableoration = 10)

figure

## Point Plot analysis

In [ ]:
x_features = ['workingday','holiday','season','weather']

mpl.rc('font',size = 10)
figure,axes = plt.subplots(nrows=4)
figure.set_size_inches(12,18)

for i,x in enumerate(x_features):
    sns.pointplot(x = 'hour', y = 'count', data=train, hue = x, ax= axes[i])
    print(f'sns.pointplot(x = hour, y = count, data=train, hue = {x}, ax= axes[{i}])')

At weather case, there are outlier at 4.
So, we delete 4 value in wether data

## Scatter plot graph with regression line analysis

In [ ]:
mpl.rc('font',size=10)
figure,axes = plt.subplots(nrows=4)
plt.tight_layout()
figure.set_size_inches(10,10)

x = ['temp','atemp','windspeed','humidity']

for i,x_ in enumerate(x):
    sns.regplot(x=x_,y='count',data=train,ax=axes[i],\
                 scatter_kws={'alpha':0.2}, line_kws={'color':'blue'})
    # scatter_kws={'alpha':0.2} : transparency degree of dots in plot, dots are not seen when 0
    # line_kws={'color':'blue'} : the line is seen as blue
    

without windspeed, data in other things is distributed evenly.

As observing windspeed, there are some missing values than others. 

So, we delete or indstaed this value

## Heat map analysis

In [ ]:
train[['temp','atemp','humidity','windspeed','count']].corr()
#corr(): return correlation value among numerical datas

In [ ]:
corrMat = train[['temp','atemp','humidity','windspeed','count']].corr() #corriation matrix
fig,axes = plt.subplots()
fig.set_size_inches(5,5)
sns.heatmap(corrMat, annot=True) # add heatmap matrix into heatmap function 
                                # and to see correlation value at the map, set annot=True
axes.set(title='Heatmap of Numerical Data')

As observing windspeed, corelation value between count and windspeed has 0.1.

This one has very lower value, which means the feature (windspeed) is seems not to help for our perdiction.